# Homework 4

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from collections import namedtuple
import itertools
import numpy as np

# Take a moment to review the data structure below (from hw03).
class FrozenMap:
  """A dict-like data structure that is immutable and hashable.

  The keys and values must be hashable and immutable too.

  Example usage:
    fm = FrozenMap(a="hello", b="world")
    
    for k in fm:
      print(fm[k])

    for k in fm.keys():
      print(fm[k])
    
    for v in fm.values():
      print(v)
    
    for k, v in fm.items():
      print(k, v)
    
    fm2 = FrozenMap(c=fm)
    print(fm2["c"]["a"])  # "hello"
    print(fm2["c"]["b"])  # "world"
  """

  def __init__(self, *args, **kwargs):
    self._dict = dict(*args, **kwargs)
    self._as_frozenset = frozenset(self._dict.items())
    self._hash = hash(self._as_frozenset)

  def __getitem__(self, key):
    return self._dict[key]

  def __iter__(self):
    return iter(self._dict)

  def __hash__(self):
    return self._hash

  def __eq__(self, other):
    return hash(self) == hash(other)

  def __str__(self):
    return str(self._dict)

  def __repr__(self):
    return f'FrozenMap({repr(self._dict)})'

  def __contains__(self, item):
    return item in self._dict

  def keys(self):
    return self._dict.keys()

  def values(self):
    return self._dict.values()

  def items(self):
    return self._dict.items()


# A random variable; name is str, dim is int
RV = namedtuple("RV", ["name", "dim"])

# rvs is a frozenset of RVs; table is a FrozenMap
# of the form {{RV: int}: float}, where the keys are
# rv joint assignments that are also FrozenMaps.
Potential = namedtuple("Potential", ["rvs", "table"])

def iter_joint_assignments(rvs):
  """Iterates over joint assignments for a list of RVs.

  Args:
    rvs: A list of RVs.

  Yields:
    assignment: A FrozenMap.
  """
  domains = []
  for rv in rvs:
    if isinstance(rv, CustomDomainRV):
      rv_domain = rv.domain
    else:
      assert isinstance(rv, RV)
      rv_domain = range(rv.dim)
    domains.append(rv_domain)
  for assignment_values in itertools.product(*domains):
    yield FrozenMap(zip(rvs, assignment_values))


def get_sub_assignment(fm, keys):
  """Given a FrozenMap and a subset of keys, return a new FrozenMap.

  Args:
    fm: A FrozenMap.
    keys: A subset of the keys in fm.

  Returns:
    new_fm: A FrozenMap.
  """
  return FrozenMap({k: fm[k] for k in keys})


def multiply_potentials(potentials):
  """Multiply potentials together.

  Args:
    potentials: A list of Potentials.
  
  Returns:
    result: A new Potential.
  """
  # Note: this implementation favors clarity over
  # efficiency. If you are curious about a more
  # efficient implementation, please ask course staff :)

  # Collect all random variables
  rvs = set()
  for pot in potentials:
    rvs |= set(pot.rvs)
  rvs = frozenset(rvs)

  # Initialize table
  table = {}

  # Iterate over assignments in the new potential
  for assignment in iter_joint_assignments(rvs):
    value = 1.
    for pot in potentials:
      sub_assignment = get_sub_assignment(assignment, pot.rvs)
      # Compute product
      value *= pot.table[sub_assignment]
    # Add value to result table
    table[assignment] = value
  # Return potential
  table = FrozenMap(table)
  return Potential(rvs, table)


def query_potential_from_rv_name_dict(rv_name_dict, potential):
  """Given a dict from RV names (strs) to assignments,
  return the corresponding value in the potential table.

  Args:
    rv_name_dict: A dict from str names to values.
    potential: A Potential.

  Returns:
    value: The float value from potential.table.
  """
  name_to_rv = {rv.name : rv for rv in potential.rvs}
  assert all(name in name_to_rv for name in rv_name_dict)
  assignment = FrozenMap({name_to_rv[n]: v for (n, v) in rv_name_dict.items()})
  return potential.table[assignment]

### New for HW04 ###

class CustomDomainRV:
  """A random variable with a custom domain.

  Example usage:
    A = CustomDomainRV("A", {"x", "y", "z"})
    print(A.domain)
    print(A.dim)
    B = CustomDomainRV("B", {(0, 0), (0, 1), (0, 2)}))
    print(B.domain)
    print(B.dim)
  """
  def __init__(self, name, domain):
    """Initialize a CustomDomainRV.

    Args:
      name: str name for the RV.
      domain: set of domain values.
    """
    self.name = name
    self.domain = set(domain)
    self.dim = len(domain)

  def __hash__(self):
    return hash((self.name, frozenset(self.domain)))

  def __repr__(self):
    return f"CustomDomainRV({self.name}, {self.domain})"

  def __str__(self):
    return self.name


HMM = namedtuple("HMM", [
    "state_rvs",  # A list of CustomDomainRVs, one per timestep
    "observation_rvs",  # A list of CustomDomainRVs, one per timestep
    "transition_potentials",  # A list of Potentials, one per transition
    "observation_potentials",  # A list of Potentials, one per timestep
    "initial_distribution"  # A single Potential
])


def condition_potential(potential, condition):
  """Given a potential and an assignment of one or more RVs,
    create a new potential representing the conditional.

  Args:
    potential: A Potential.
    condition: A FrozenMap from RVs to values.

  Returns:
    conditional_potential: A Potential.
  """
  new_table = {}
  conditioned_rvs = set(condition.keys())
  new_rvs = frozenset(potential.rvs - conditioned_rvs)
  for assignment, value in potential.table.items():
    # Check if assignment is consistent with condition
    consistent = True
    for v, val in assignment.items():
      if v in conditioned_rvs and condition[v] != val:
        consistent = False
        break
    if not consistent:
      continue
    # Add
    sub_assignment = get_sub_assignment(assignment, new_rvs)
    assert sub_assignment not in new_table
    new_table[sub_assignment] = value
  return Potential(new_rvs, FrozenMap(new_table))


def normalize_potential(potential):
  """Normalize the values of a potential so they sum to 1.

  Args:
    potential: A Potential.

  Returns:
    new_potential: A Potential.
  """
  denom = sum(potential.table.values())
  if denom == 0:
    print("WARNING: tried to normalize an all-zeros potential.")
    return potential
  new_table = {}
  for assignment, value in potential.table.items():
    new_table[assignment] = value / denom
  return Potential(potential.rvs, FrozenMap(new_table))


def max_potential(potential, rvs=None):
  """Create a new potential where each rv has been maximized out.
  
  If rvs is None, then this function returns the max
  over all assignments in the potential.

  Args:
    potential: A Potential.
    rvs: A set of random variables in the potential or None.
  
  Returns:
    new_potential: A Potential.
  """
  if rvs is None:
    return max(potential.table.values())

  new_rvs = frozenset({rv for rv in potential.rvs if rv not in rvs})
  new_table = {}
  for assignment, value in potential.table.items():
    sub_assignment = get_sub_assignment(assignment, new_rvs)
    if sub_assignment not in new_table:
      new_table[sub_assignment] = value
    else:
      new_table[sub_assignment] = max(new_table[sub_assignment], value)
  new_table = FrozenMap(new_table)
  return Potential(new_rvs, new_table)


def argmax_potential(potential, rvs=None):
  """Create a new "potential" where the values are FrozenMap assignments
  of the given rvs that lead to maximum values.

  If rvs is None, then this function returns the argmax assignment
  over all assignments in the potential.
  
  Args:
    potential: A Potential.
    rvs: A set of random variables in the potential or None.
  
  Returns:
    new_potential: A Potential.
  """
  if rvs is None:
    return max(potential.table.keys(), key=lambda a: potential.table[a])

  new_rvs = frozenset({rv for rv in potential.rvs if rv not in rvs})
  new_table = {}
  max_values_table = {}
  for assignment, value in potential.table.items():
    sub_assignment = get_sub_assignment(assignment, new_rvs)
    if sub_assignment not in new_table or max_values_table[sub_assignment] < value:
      new_table[sub_assignment] = get_sub_assignment(assignment, rvs)
      max_values_table[sub_assignment] = value
  new_table = FrozenMap(new_table)
  return Potential(new_rvs, new_table)

def create_hmm(obstacle_map, num_timesteps):
  """Converts a map into an HMM.
  
  Creates the state RVs, the observation RVs, the transition potentials,
  and the observation potentials.

  Args:
    obstacle_map: A list of lists of ints; see example and
        description in `create_state_variable`.
    num_timesteps: An int number of timesteps, must be >= 1.

  Returns:
    problem: An HMM.
  """
  assert num_timesteps >= 1

  # Create variables
  state_rvs = []
  observation_rvs = []
  for t in range(num_timesteps):
    state_t = create_state_variable(obstacle_map, f"state{t}")
    obs_t = create_observation_variable(f"observation{t}")
    state_rvs.append(state_t)
    observation_rvs.append(obs_t)

  # Create observation potentials
  observation_potentials = []
  for state, obs in zip(state_rvs, observation_rvs):
    obs_pot_t = create_observation_potential(obstacle_map, state, obs)
    observation_potentials.append(obs_pot_t)

  # Create transition potentials
  transition_potentials = []
  if num_timesteps > 1:
    for state_t, state_t1 in zip(state_rvs[:-1], state_rvs[1:]):
      trans_pot_t = create_transition_potential(obstacle_map, state_t, state_t1)
      transition_potentials.append(trans_pot_t)

  # Create (uniform) initial state distribution
  table = {}
  init_state_rv = state_rvs[0]
  for s in init_state_rv.domain:
    table[FrozenMap({init_state_rv: s})] = 1.0/init_state_rv.dim
  table = FrozenMap(table)
  initial_distribution = Potential(frozenset({init_state_rv}), table)
  
  return HMM(state_rvs, observation_rvs, transition_potentials,
             observation_potentials, initial_distribution)



## Problems

### Warmup 1
Write a function that creates a CustomDomainRV named "Weather" that has 3 values in its domain. See the docstring for the description. You will also need to look at the documentation in colab for how to create a CustomDomainRV.

For reference, our solution is **1** lines of code.

In [ ]:
def warmup1():
  """Creates a CustomDomainRV.

  The RV should be called "Weather", and have 3 values in its domain: Sunny,
  Cloudy and Raining.

  Returns:
    v: A CustomDomainRV
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  warmup1_rv = warmup1()
  assert warmup1_rv.name == "Weather"
  assert warmup1_rv.dim == 3
  assert "Sunny" in warmup1_rv.domain
  assert "Cloudy" in warmup1_rv.domain
  assert "Raining" in warmup1_rv.domain
  
print('Tests passed.')

### Warmup 2
Write a function that creates a uniform Potential between two
    CustomDomainRVs. Remember that a potential contains a frozenset of random
    variables, and a FrozenMap of a dict of assignments to values. This is not
    that different from the last assignment, but the assignments are now
    CustomDomainRVs to their custom domain values instead of RVs mapped to numbers.

For reference, our solution is **10** lines of code.

In [ ]:
def warmup2(rv1, rv2):
  """Create a Potential between CustomDomainRVs, where all assignments have
  the same value. 

  Input:
    rv1: A CustomDomainRV
    rv2: A CustomDomainRV

  Returns:
    p: A Potential
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  rv1 = CustomDomainRV("fruit", {"apple", "banana"})
  rv2 = CustomDomainRV("status", {"ripe", "rotten"})
  warmup2_pot = warmup2(rv1, rv2)
  assert isinstance(warmup2_pot, Potential)
  assert warmup2_pot.rvs == frozenset({rv1, rv2})
  assert query_potential_from_rv_name_dict({"fruit": "apple", "status": "ripe"}, warmup2_pot) == 0.25
  assert query_potential_from_rv_name_dict({"fruit": "apple", "status": "rotten"}, warmup2_pot) == 0.25
  assert query_potential_from_rv_name_dict({"fruit": "banana", "status": "ripe"}, warmup2_pot) == 0.25
  assert query_potential_from_rv_name_dict({"fruit": "banana", "status": "rotten"}, warmup2_pot) == 0.25
  
print('Tests passed.')

### State variable creation
Write a function that creates a random variable for a state at a given time step in an obstacle HMM. See docstring for description.

For reference, our solution is **3** lines of code.

In [ ]:
def create_state_variable(obstacle_map, name):
  """Creates a CustomDomainRV for the HMM state.

  The state can be any position in the map.

  Example map:
    obstacle_map = [
        [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    ]

  Zeros are obstacles and ones are free positions.

  The domain of the state variable should be a list of (row, col)
  indices into the map. Only free positions (not obstacles) should
  be included in the domain of the state variable.

  Args:
    obstacle_map: A list of lists of ints, see example above.
    name: A str name for the state variable.

  Returns:
    state_var: A CustomDomainRV as described above.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  map1 = [[1, 0, 1],
          [0, 1, 1]]
  rv = create_state_variable(map1, "current_state")
  assert rv.name == "current_state"
  assert rv.dim == 4
  assert rv.domain == {(0, 0), (0, 2), (1, 1), (1, 2)}
  
print('Tests passed.')

### Transition potential creation
Write a function that creates a potential for the transition distribution between states $s_{t}$ and $s_{t+1}$.

For reference, our solution is **18** lines of code.

In [ ]:
def create_transition_potential(obstacle_map, st, st1):
  """Write a function to create a potential for the transition from state s_t
  to s_{t+1}, in an HMM that corresponds to the map.

  Transitions are uniformly distributed amongst the robot's current
  location and the neighboring free (not obstacle) locations, where
  neighboring = 4 cardinal directions (up, down, left, right).

  Args:
    st: An CustomDomainRV representing the state at time t.
    st1: An CustomDomainRV representing the state at time t+1.
    obstacle_map: A list of lists of ints;
        see example and description in `create_state_variable`.

  Returns:
    potential: A Potential for the transition between st and st1.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  map1 = [[1, 1, 1],
          [0, 1, 0]]
  s0 = create_state_variable(map1, "state_0")
  s1 = create_state_variable(map1, "state_1")
  pot = create_transition_potential(map1, s0, s1)
  assert pot.rvs == frozenset({s0, s1})
  assert query_potential_from_rv_name_dict({"state_0": (0, 0), "state_1": (0, 0)}, pot) == 0.5
  assert query_potential_from_rv_name_dict({"state_0": (0, 0), "state_1": (0, 1)}, pot) == 0.5
  assert query_potential_from_rv_name_dict({"state_0": (0, 0), "state_1": (1, 1)}, pot) == 0.
  assert query_potential_from_rv_name_dict({"state_0": (0, 1), "state_1": (1, 1)}, pot) == 0.25
  assert query_potential_from_rv_name_dict({"state_0": (0, 1), "state_1": (0, 1)}, pot) == 0.25
  assert query_potential_from_rv_name_dict({"state_0": (0, 1), "state_1": (0, 2)}, pot) == 0.25
  assert query_potential_from_rv_name_dict({"state_0": (0, 1), "state_1": (0, 0)}, pot) == 0.25
  assert query_potential_from_rv_name_dict({"state_0": (1, 1), "state_1": (0, 0)}, pot) == 0.
  assert query_potential_from_rv_name_dict({"state_0": (1, 1), "state_1": (0, 1)}, pot) == 0.5
  
print('Tests passed.')

### Observation variable creation
Write a function that creates a random variable for an observation at a given time step in an obstacle HMM. See docstring for description.

For reference, our solution is **2** lines of code.

In [ ]:
def create_observation_variable(name):
  """Creates a CustomDomainRV for the HMM observation with the given name.

  Observations are a 4-tuple that list which directions have obstacles,
  in order [N E S W], with a 0 for an obstacle and 1 for no
  obstacle. The observations are perfect and error-free. For instance,
  in the following map:

    obstacle_map = [
        [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    ]

  the observation for the top left location would always be (0, 1, 0, 0).

  The domain of the observation variable should be a list of 4-tuples.

  Args:
    name: A str name for the variable.

  Returns:
    zt: A CustomDomainRV as described above.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  rv = create_observation_variable("obs")
  assert rv.name == "obs"
  assert rv.dim == 2**4
  assert rv.domain == {
    (0, 0, 0, 0), (1, 0, 0, 0), (0, 1, 0, 0), (1, 1, 0, 0),
    (0, 0, 1, 0), (1, 0, 1, 0), (0, 1, 1, 0), (1, 1, 1, 0),
    (0, 0, 0, 1), (1, 0, 0, 1), (0, 1, 0, 1), (1, 1, 0, 1),
    (0, 0, 1, 1), (1, 0, 1, 1), (0, 1, 1, 1), (1, 1, 1, 1),
    }
  
print('Tests passed.')

### Observation potential creation
Write a function that creates a potential for the observation distribution between $s_{t}$ and $z_{t}$.

For reference, our solution is **18** lines of code.

In [ ]:
def create_observation_potential(obstacle_map, state_rv, observation_rv):
  """Write a function to create a potential between state state_rv
  and observation_rv in an HMM that corresponds to the map.

  You can assume that state_rv was created by `create_state_variable`
  and observation_rv was created by `create_observation_variable`.

  See `create_observation_variable` for a description of the
  observation model. Recall the order is [N E S W].

  Args:
    obstacle_map: A list of lists of ints;
        see example and description in `create_state_variable`.
    state_rv: An CustomDomainRV representing the state at time t.
    observation_rv: An CustomDomainRV representing the observation at time t.

  Returns:
    potential: A Potential for the distribution between st and zt.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  map1 = [[1, 1, 1],
          [0, 1, 0]]
  s0 = create_state_variable(map1, "state_0")
  z0 = create_observation_variable("obs_0")
  pot = create_observation_potential(map1, s0, z0)
  assert pot.rvs == frozenset({s0, z0})
  assert query_potential_from_rv_name_dict({"state_0": (0, 0), "obs_0": (0, 1, 0, 0)}, pot) == 1.
  assert query_potential_from_rv_name_dict({"state_0": (0, 0), "obs_0": (0, 0, 1, 0)}, pot) == 0.
  assert query_potential_from_rv_name_dict({"state_0": (0, 1), "obs_0": (0, 1, 1, 1)}, pot) == 1.
  assert query_potential_from_rv_name_dict({"state_0": (0, 2), "obs_0": (0, 0, 0, 1)}, pot) == 1.
  assert query_potential_from_rv_name_dict({"state_0": (1, 1), "obs_0": (1, 0, 0, 0)}, pot) == 1.
  
print('Tests passed.')

### Viterbi Decoding
Complete the implementation of Viterbi decoding.

For reference, our solution is **39** lines of code.

In [ ]:
def run_viterbi(hmm, observations):
  """Run the Viterbi algorithm to compute the most likely state
  assignments given observations.

  Args:
    hmm: An HMM.
    observations: A list of observation values.

  Returns:
    most_likely_states: A list of state values.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
  map1 = [[1, 1, 1],
          [0, 1, 0]]
  observations1 = [(0, 1, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1)]
  hmm1 = create_hmm(map1, len(observations1))
  most_likely_states1 = run_viterbi(hmm1, observations1)
  assert most_likely_states1 == [(0, 0), (0, 1), (0, 1)]
  
  map2 = [[1, 1, 1, 1, 1]]
  observations2 = [(0, 1, 0, 1), (0, 1, 0, 0), (0, 1, 0, 1),
                   (0, 1, 0, 1), (0, 1, 0, 1), (0, 0, 0, 1)]
  hmm2 = create_hmm(map2, len(observations2))
  most_likely_states2 = run_viterbi(hmm2, observations2)
  assert most_likely_states2 == [(0, 1), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
  
print('Tests passed.')